In [52]:
from gbmi.exp_modular_fine_tuning.train import MODULAR_ADDITION_113_CLOCK_CONFIG
from gbmi.exp_modular_fine_tuning.train import ModularFineTuningTrainingWrapper
from gbmi.model import train_or_load_model
import torch
import einops
from torch import tensor
from math import *
device = 'cuda'

In [53]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [54]:
config = MODULAR_ADDITION_113_CLOCK_CONFIG
runtime, model = train_or_load_model(config)
model.to(device)

wandb:   1 of 1 files downloaded.  
wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


Moving model to device:  cuda


HookedTransformer(
  (embed): Embed()
  (hook_embed): HookPoint()
  (pos_embed): PosEmbed()
  (hook_pos_embed): HookPoint()
  (blocks): ModuleList(
    (0): TransformerBlock(
      (ln1): Identity()
      (ln2): Identity()
      (attn): Attention(
        (hook_k): HookPoint()
        (hook_q): HookPoint()
        (hook_v): HookPoint()
        (hook_z): HookPoint()
        (hook_attn_scores): HookPoint()
        (hook_pattern): HookPoint()
        (hook_result): HookPoint()
      )
      (mlp): MLP(
        (hook_pre): HookPoint()
        (hook_post): HookPoint()
      )
      (hook_attn_in): HookPoint()
      (hook_q_input): HookPoint()
      (hook_k_input): HookPoint()
      (hook_v_input): HookPoint()
      (hook_mlp_in): HookPoint()
      (hook_attn_out): HookPoint()
      (hook_mlp_out): HookPoint()
      (hook_resid_pre): HookPoint()
      (hook_resid_mid): HookPoint()
      (hook_resid_post): HookPoint()
    )
  )
  (unembed): Unembed()
)

In [55]:
for param in model.parameters():
    param.requires_grad = False
embedding_matrix = model.embed.W_E.clone()
embedding_matrix.requires_grad = False
model.embed = torch.nn.Identity()

In [56]:
def loss_fn(logits,labels):
    print(logits.shape)
    if len(logits.shape)==3:
        logits = logits[:,:, -1]
    logits = logits.to(torch.float64)
    log_probs = logits.log_softmax(dim=-1)
    correct_log_probs = log_probs.gather(dim=-1, index=labels[:, None])[:, 0]
    return -correct_log_probs.mean()

In [81]:
p = 113
q = 2*p
class DifferentModClock(torch.nn.Module):
    def __init__(self,q):
        super(DifferentModClock,self).__init__()
        self.W_e = torch.nn.Parameter(-0.5/sqrt(p+1) + torch.rand((q+1,p+1))/sqrt(p+1)).to(device)
        self.W_u = torch.nn.Parameter(-0.5/sqrt(p) + torch.rand((q,p))/sqrt(p)).to(device)
        self.params = torch.nn.ParameterList([self.W_e,self.W_u])
    def forward(self,x):
        z = torch.nn.functional.one_hot(x.to(device)).to(torch.float)
        
        y = z @ self.W_e @ embedding_matrix
        if(len(y.shape)!=3):
            y = y.unsqueeze(0)
        return self.W_u @ (einops.rearrange(model(y),"b d p -> b p d"))
clock = DifferentModClock(q)

print(clock(torch.tensor([[1,2,226],[2,5,226]])))

tensor([[[-6.1755, -0.9262,  0.4199],
         [ 8.8307,  2.8965,  1.1911],
         [ 0.6976,  0.2317,  0.9714],
         ...,
         [ 4.6736,  2.9706,  0.2907],
         [-9.5380, -1.7593, -1.5765],
         [-1.0700, -0.4691,  0.0631]],

        [[-0.3219, -1.7656, -1.2217],
         [ 4.9145,  2.5245,  0.6075],
         [ 0.1035,  0.4361,  0.5748],
         ...,
         [ 4.8511,  3.5585,  0.9854],
         [-7.0302, -2.4302, -0.7050],
         [-2.3406,  0.6992,  2.8796]]], device='cuda:0',
       grad_fn=<CloneBackward0>)


In [82]:
a_vector = einops.repeat(torch.arange(q), "i -> (i j)", j=q)
b_vector = einops.repeat(torch.arange(q), "j -> (i j)", i=q)
equals_vector = einops.repeat(torch.tensor(q), " -> (i j)", i=q, j=q)
dataset = torch.stack([a_vector, b_vector, equals_vector], dim=1).to(device)
print(dataset.shape)

torch.Size([51076, 3])


In [83]:
optim =torch.optim.AdamW(clock.parameters())

In [84]:
clock.W_e.retain_grad()
def run_batch(x):
        labels = (x[:, 0] + x[:, 1]) % q

        y_preds = clock(x)

        loss = loss_fn(y_preds, labels)
        print(loss)
        loss.backward()
        optim.step()
        optim.zero_grad()
        return loss


In [90]:
run_batch(dataset)

torch.Size([51076, 226, 3])
tensor(11.5251, device='cuda:0', dtype=torch.float64, grad_fn=<NegBackward0>)


tensor(11.5251, device='cuda:0', dtype=torch.float64, grad_fn=<NegBackward0>)